In [2]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi 

In [9]:
data = pd.read_csv('./data/nesarc_pds.csv', low_memory=False)

In [16]:
# convert into numeric types
data['S3AQ3B1'] = data['S3AQ3B1'].convert_objects(convert_numeric=True)
data['S3AQ3C1'] = data['S3AQ3C1'].convert_objects(convert_numeric=True)
data['CHECK321'] = data['CHECK321'].convert_objects(convert_numeric=True)

In [17]:
#subset data to young adults age 18 to 25 who have smoked in the past 12 months
sub1=data[(data['AGE']>=18) & (data['AGE']<=25) & (data['CHECK321']==1)]

In [18]:
#SETTING MISSING DATA
sub1['S3AQ3B1']=sub1['S3AQ3B1'].replace(9, numpy.nan)
sub1['S3AQ3C1']=sub1['S3AQ3C1'].replace(99, numpy.nan)

In [19]:
#recoding number of days smoked in the past month
recode1 = {1: 30, 2: 22, 3: 14, 4: 5, 5: 2.5, 6: 1}
sub1['USFREQMO']= sub1['S3AQ3B1'].map(recode1)

In [20]:
#converting new variable USFREQMMO to numeric
sub1['USFREQMO']= sub1['USFREQMO'].convert_objects(convert_numeric=True)

In [21]:
# Creating a secondary variable multiplying the days smoked/month and the number of cig/per day
sub1['NUMCIGMO_EST']=sub1['USFREQMO'] * sub1['S3AQ3C1']

In [22]:
sub1['NUMCIGMO_EST']= sub1['NUMCIGMO_EST'].convert_objects(convert_numeric=True)
ct1 = sub1.groupby('NUMCIGMO_EST').size()
print (ct1)

NUMCIGMO_EST
1.0        29
2.0        14
2.5        11
3.0        12
4.0         2
5.0        34
6.0         1
7.5        12
8.0         1
10.0       38
12.5        9
14.0        3
15.0       14
17.5        1
20.0       13
22.0        4
24.0        1
25.0       14
28.0       17
30.0       25
35.0        2
42.0       19
44.0        9
50.0        7
56.0       15
60.0       28
66.0       14
70.0       22
84.0        3
88.0        6
         ... 
140.0      10
150.0     108
154.0       3
176.0       3
180.0      47
210.0      39
220.0      12
240.0      36
270.0       6
280.0       1
300.0     350
330.0       4
360.0      25
390.0       7
420.0       2
450.0      97
480.0       5
510.0       2
540.0       3
570.0       1
600.0     357
750.0      13
810.0       1
840.0       1
900.0      38
1050.0      1
1200.0     29
1800.0      2
2400.0      1
2940.0      1
dtype: int64


In [23]:
# using ols function for calculating the F-statistic and associated p value
model1 = smf.ols(formula='NUMCIGMO_EST ~ C(MAJORDEPLIFE)', data=sub1)
results1 = model1.fit()
print (results1.summary())

                            OLS Regression Results                            
Dep. Variable:           NUMCIGMO_EST   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     3.550
Date:                Sat, 12 Nov 2016   Prob (F-statistic):             0.0597
Time:                        13:39:19   Log-Likelihood:                -11934.
No. Observations:                1697   AIC:                         2.387e+04
Df Residuals:                    1695   BIC:                         2.388e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------------
Intercept              312.8380 

In [24]:
sub2 = sub1[['NUMCIGMO_EST', 'MAJORDEPLIFE']].dropna()
print ('means for numcigmo_est by major depression status')
m1= sub2.groupby('MAJORDEPLIFE').mean()
print (m1)

means for numcigmo_est by major depression status
              NUMCIGMO_EST
MAJORDEPLIFE              
0               312.837989
1               341.375000


In [25]:
print ('standard deviations for numcigmo_est by major depression status')
sd1 = sub2.groupby('MAJORDEPLIFE').std()
print (sd1)

standard deviations for numcigmo_est by major depression status
              NUMCIGMO_EST
MAJORDEPLIFE              
0               269.002344
1               288.495118


In [26]:
#i will call it sub3
sub3 = sub1[['NUMCIGMO_EST', 'ETHRACE2A']].dropna()

model2 = smf.ols(formula='NUMCIGMO_EST ~ C(ETHRACE2A)', data=sub3).fit()
print (model2.summary())

                            OLS Regression Results                            
Dep. Variable:           NUMCIGMO_EST   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.052
Method:                 Least Squares   F-statistic:                     24.40
Date:                Sat, 12 Nov 2016   Prob (F-statistic):           1.18e-19
Time:                        13:39:57   Log-Likelihood:                -11888.
No. Observations:                1697   AIC:                         2.379e+04
Df Residuals:                    1692   BIC:                         2.381e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
Intercept           368.7865      8.22

In [27]:
print ('means for numcigmo_est by major depression status')
m2= sub3.groupby('ETHRACE2A').mean()
print (m2)

means for numcigmo_est by major depression status
           NUMCIGMO_EST
ETHRACE2A              
1            368.786528
2            259.273810
3            310.988095
4            244.258621
5            219.758258


In [28]:
print ('standard deviations for numcigmo_est by major depression status')
sd2 = sub3.groupby('ETHRACE2A').std()
print (sd2)

standard deviations for numcigmo_est by major depression status
           NUMCIGMO_EST
ETHRACE2A              
1            281.430730
2            278.677392
3            260.116964
4            195.076441
5            220.859365


In [29]:
mc1 = multi.MultiComparison(sub3['NUMCIGMO_EST'], sub3['ETHRACE2A'])
res1 = mc1.tukeyhsd()
print(res1.summary())

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2  meandiff   lower     upper   reject
--------------------------------------------------
  1      2    -109.5127 -164.6441  -54.3814  True 
  1      3     -57.7984 -172.5914  56.9945  False 
  1      4    -124.5279 -222.9229  -26.1329  True 
  1      5    -149.0283  -194.89  -103.1665  True 
  2      3     51.7143   -71.6021  175.0307 False 
  2      4     -15.0152  -123.233  93.2026  False 
  2      5     -39.5156 -103.8025  24.7714  False 
  3      4     -66.7295 -214.5437  81.0848  False 
  3      5     -91.2298 -210.6902  28.2305  False 
  4      5     -24.5004 -128.3027   79.302  False 
--------------------------------------------------
